# Procedimentos como objetos

Este notebook é baseado nos capítulos 05 e 07 do livro [Python Fluente](https://novatec.com.br/livros/pythonfluente/) do autor [Luciano Ramalho](https://www.linkedin.com/in/lucianoramalho/). Os exemplos originais podem ser encontrados no [repositório do livro](https://github.com/fluentpython).

Em Python, tudo é um objeto ou uma referência a um objeto. Procedimentos são definidos como objetos, que podem ser referenciados tanto por seu nome como por outras referências:

In [1]:
def triplo(numero):
    return numero * 3

print(triplo(4))
triplo_tb = triplo
print(triplo_tb(4))

12
12


In [2]:
type(triplo)

function

Tratar procedimentos como objetos abre ainda outras possibilidades importantes:

* Associado a uma referência ou a um elemento de uma estrutura de dados:

In [3]:
def print_hello():
    print("Hello ", end="")

def print_world():
    print("world", end="")

hello_world = [print_hello, print_world, print_world]
for print_ in hello_world:
    print_()

Hello worldworld

* Ser passado como argumento a uma função:

In [4]:
def duplo(numero):
    return numero * 2

def multiplo(procedimento, numero):
    return procedimento(numero)

print(multiplo(duplo, 4))
print(multiplo(triplo, 4))

8
12


In [5]:
list(map(len, "Esta é uma string".split()))

[4, 1, 3, 6]

In [6]:
list(map(duplo, [1, 3, 6, 10]))

[2, 6, 12, 20]

* Ser retornado por um procedimento:

In [7]:
from random import choice
def random_print(lista):
    return choice(lista)

print_ = random_print(hello_world)
print_()

Hello 

In [8]:
from random import randint
def criador_de_procedimentos(maximo):
    fator = randint(1,maximo)
    def multiplo_aleatorio(numero):
        return numero * fator
    return multiplo_aleatorio

multiplicador1 = criador_de_procedimentos(5)
multiplicador2 = criador_de_procedimentos(10)

print(" ".join(str(multiplicador1(i)) for i in range(10)))
print(" ".join(str(multiplicador2(i)) for i in range(10)))

0 5 10 15 20 25 30 35 40 45
0 2 4 6 8 10 12 14 16 18


### Procedimentos de ordem superior

Procedimentos que aceitam outros procedimentos como argumentos e/ou retornam procedimentos são conhecidos como **procedimentos de ordem superior**. 

É assim que funciona o parâmetro ```key``` presente em muitos procedimentos embutidos do Python, como ```sorted```:

In [16]:
frutas = ["morango", "figo", "maçã", "cereja", "framboesa", "banana"]

In [19]:
tamanhos = list(map(len, frutas))
print(tamanhos)

[7, 4, 4, 6, 9, 6]


In [20]:
sorted(tamanhos)

[4, 4, 6, 6, 7, 9]

In [9]:
sorted(frutas, key=len)

['figo', 'maçã', 'cereja', 'banana', 'morango', 'framboesa']

In [21]:
def len_lex(palavra):
    return (len(palavra), palavra)

In [22]:
sorted(frutas, key=len_lex)

['figo', 'maçã', 'banana', 'cereja', 'morango', 'framboesa']

In [23]:
sorted(frutas, key=(len, lambda x: x))

TypeError: 'tuple' object is not callable

In [25]:
sorted(frutas, key=[len, lambda x: x])

TypeError: 'list' object is not callable

Outro exemplo interessante é o procedimento ```partial```, disponível no módulo ```functools```. 

Este procedimento permite pré-definir o valor de determinados parâmetros de um outro procedimento, sendo bastante útil quando queremos especializar o comportamento de um procedimento mais geral.

O exemplo abaixo mostra como especializar o procedimento ```mul```, que originalmente multiplica uma lista de parâmetros:

In [10]:
from operator import mul
from functools import partial

duplo = partial(mul, 2)
triplo = partial(mul, 3)

print(duplo(4))
print(triplo(4))

8
12


## De parâmetros posicionais a parâmetros exclusivamente nomeados

Em Python, um procedimento pode apresentar parâmetros de três diferentes categorias:
1. Posicional
2. Nomeado
3. Exclusivamente nomeado

Parâmetros das categorias 1 e 2 são comuns em outras linguagens, sendo importante ressaltar apenas que um parâmetro posicional pode ser passado como se fosse nomeado, ou um parâmetro nomeado ser passado como posicional:

In [11]:
def procedimento(posicional1, posicional2, nomeado1=3, nomeado2=4):
    print(
        """
        posicional1: {}, 
        posicional2: {}, 
        nomeado1: {}, 
        nomeado2: {}""".format(
            posicional1, posicional2, nomeado1, nomeado2)
        )

* Parâmetros nomeados apresentam valores-padrão, não precisando ser informados a cada chamada:

In [12]:
procedimento(1, 2)

posicional1: 1, posicional2: 2, nomeado1: 3, nomeado2: 4


In [32]:
procedimento(1)

TypeError: procedimento() missing 1 required positional argument: 'posicional2'

* Obviamente, parâmetros nomeados também podem ser informados, caso não se deseje usar seus valores-padrão. É possível fazer isso usando o nome do parâmetro ou não:

In [13]:
procedimento(1, 5, 6)
procedimento(1, 5, nomeado2=10)

posicional1: 1, posicional2: 5, nomeado1: 6, nomeado2: 4
posicional1: 1, posicional2: 5, nomeado1: 3, nomeado2: 10


* Também é possível tratar um parâmetro posicional como nomeado, sendo possível informá-lo fora de ordem:

In [14]:
procedimento(1, nomeado2=10, posicional2=5)

posicional1: 1, posicional2: 5, nomeado1: 3, nomeado2: 10


* Não é possível, no entanto, informar um parâmetro posicional utilizando seu nome se já houver sido informado um parâmetro posicional que também possa ser interpretado como ele:

In [15]:
procedimento(1, 3, nomeado2=10, posicional2=5)

TypeError: procedimento() got multiple values for argument 'posicional2'

* Também não é possível informar um parâmetro originalmente nomeado antes de um parâmetro originalmente posicional:

In [33]:
procedimento(nomeado1=30, 10, 20)

SyntaxError: positional argument follows keyword argument (<ipython-input-33-ae3d973d63cc>, line 1)

* Um recurso importante em Python é a possibilidade de ter uma lista variável de parâmetros. Pra isso, basta usar um parâmetro ```*args```:

In [35]:
def print_all(titulo, ok="", *args):
    print(titulo)
    print(" ".join(str(arg) for arg in args))

print_all("Título", "Esta", "é", "uma", "frase")

Título
é uma frase


In [36]:
print_all(ok="Não", "Título", "Esta é uma frase".split())

SyntaxError: positional argument follows keyword argument (<ipython-input-36-8307c4f60465>, line 1)

In [37]:
print_all("Título", "Esta é uma frase".split())

Título



### Parâmetros exclusivamente nomeados

Um recurso pensado no Python 3 é a presença de parâmetros exclusivamente nomeados. 

Estes parâmetros se diferenciam dos demais porque só podem ser informados de forma nomeada e podem não apresentar valor padrão.

Para dizer que um parâmetro é exclusivamente nomeado, ele deve aparecer após o parâmetro ```*args```:

In [38]:
def print_all(titulo, *args, separador=" "):
    print(titulo)
    print(separador.join(str(arg) for arg in args))

print_all("Título", "Esta", "é", "uma", "frase")
print_all("Título 2", "Esta", "é", "outra", "frase", separador=";")

Título
Esta é uma frase
Título 2
Esta;é;outra;frase


In [39]:
def print_all(titulo, end="\n", *args, separador=" "):
    print(titulo)
    print(separador.join(str(arg) for arg in args), end=end)

print_all("Título", "Esta", "é", "uma", "frase")
print_all("Título 2", "Esta", "é", "outra", "frase", separador=";")

Título
é uma fraseEstaTítulo 2
é;outra;fraseEsta

In [40]:
def print_any(titulo, *args, separador):
    print(titulo)
    print(separador.join(str(arg) for arg in args))

print_any("Título", "Esta", "é", "uma", "frase")

TypeError: print_any() missing 1 required keyword-only argument: 'separador'

Assim como existe o parâmetro ```*args```, é possível usar o parâmetro ```**kwargs``` para lidar com uma quantidade variável de parâmetros exclusivamente nomeados:

In [41]:
def print_pairs(titulo, *args, separador="\t", **kwargs):
    print(titulo)
    print(separador.join("{}: {}".format(key, value) for key, value in kwargs.items()))

print_pairs("Título", a="Esta", b="é", c="uma", d="frase")

Título
a: Esta	b: é	c: uma	d: frase


In [42]:
def print_pairs(titulo, *args, separador="\t"):
    print(titulo)
    print(separador.join([str(arg) for arg in args]))

print_pairs("Título", a="Esta", b="é", c="uma", d="frase")

TypeError: print_pairs() got an unexpected keyword argument 'a'

In [43]:
def print_pairs(titulo, *, separador="\t", **kwargs):
    print(titulo)
    print(separador.join("{}: {}".format(key, value) for key, value in kwargs.items()))

print_pairs("Título", a="Esta", b="é", c="uma", d="frase")

Título
a: Esta	b: é	c: uma	d: frase


In [44]:
print_pairs("Título", 1, 2, a="Esta", b="é", c="uma", d="frase")

TypeError: print_pairs() takes 1 positional argument but 3 were given

* Note que o parâmetro ```**kwargs``` é um **dicionário**!

## Decoradores

Uma ferramenta importante no dia a dia de programadores é o uso de decoradores. 

Eles basicamente conferem alguma funcionalidade adicional a um objeto já existente.

Em Python, decoradores podem ser implementados de forma simples, como procedimentos invocados através do operador ```@```. 

In [45]:
def borda(procedimento_decorado):
    def print_borda(*args):
        print("=" * 10)
        procedimento_decorado(*args)
        print("-" * 10)
    return print_borda

@borda
def print_one(titulo, argumento):
    print("{}\n{}".format(titulo, argumento))

print_one("Título 1", "Este é um procedimento decorado!")
print_one("Título 2", "Este é outro procedimento decorado!")

Título 1
Este é um procedimento decorado!
----------
Título 2
Este é outro procedimento decorado!
----------


Na prática, a função ```borda``` encapsula a função ```print_one```, equivalendo à expressão:

```python3
print_one = borda(print_one)
``` 

Assim, cada vez que ```print_one``` é chamado, ```borda``` controla sua execução. 

É importante destacar que o código de um procedimento usado como decorador é interpretado assim que o módulo é carregado, mas o procedimento retornado só é executado quando o procedimento decorador é invocado:

In [46]:
def procedimento_decorador(procedimento_decorado):
    print("Roda assim que é carregado..")
    def procedimento_retornado(*args):
        print("Rodando o procedimento decorado..")
        procedimento_decorado(*args)
        print("Saindo do procedimento decorado..")
    print("Saindo do procedimento decorador..")
    return procedimento_retornado

@procedimento_decorador
def print_opa():
    print("Opa!")

Roda assim que é carregado..
Saindo do procedimento decorador..


In [47]:
print_opa()

Rodando o procedimento decorado..
Opa!
Saindo do procedimento decorado..


### Decoradores aninhados e parametrizados

Assim como é possível ter procedimentos aninhados, também é possível ter decoradores aninhados:

In [48]:
def decorador_par(decorado):
    def decorador(*args):
        print("[{}] ".format("Ímpar" if args[0] % 2 else "Par"), end="")
        decorado(*args)
    return decorador

def decorador_rombo(decorado):
    def decorador(*args):
        if not (args[0] % 10):
            print("[De rombo!] ", end="")
        decorado(*args)
    return decorador

@decorador_rombo
@decorador_par
def print_carneiros(numero):
    print("{} carneiro{}".format(numero, "s" if numero != 1 else ""))

In [49]:
for i in range(11):
    print_carneiros(i)

[De rombo!] [Par] 0 carneiros
[Ímpar] 1 carneiro
[Par] 2 carneiros
[Ímpar] 3 carneiros
[Par] 4 carneiros
[Ímpar] 5 carneiros
[Par] 6 carneiros
[Ímpar] 7 carneiros
[Par] 8 carneiros
[Ímpar] 9 carneiros
[De rombo!] [Par] 10 carneiros


O código acima é equivalente à expressão:

```python3
print_carneiros = decorador_rombo(decorador_par(print_carneiros))
```

Também é possível aceitar parâmetros ao definir/utilizar um decorador:

In [50]:
def decorador_loop(maximo=5):
    def decorador(decorado):
        def decorador_retornado(*args):
            for i in range(maximo):
                decorado(*args)
        return decorador_retornado
    return decorador

@decorador_loop()
def print_random5(minimo, maximo):
    print(randint(minimo, maximo))

@decorador_loop(10)
def print_random10(minimo, maximo):
    print(randint(minimo, maximo))

In [51]:
print_random5(3,10)

9
9
7
6
10


In [52]:
print_random10(3,10)

10
8
4
7
4
5
5
8
8
10
